In [ ]:
!pip install recommenders cornac==2.3.0 ranx

In [ ]:
!add-apt-repository ppa:ubuntu-toolchain-r/test
!apt-get update
!apt-get install --only-upgrade libstdc++6

## Check Hardware Software Specification

In [1]:
!python --version

Python 3.11.12


In [2]:
!pip --version

pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)


In [3]:
!cat /etc/*release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.4 LTS"
PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [4]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 7
microcode	: 0xffffffff
cpu MHz		: 2200.228
cache size	: 39424 KB
physical id	: 0
siblings	: 12
core id		: 0
cpu cores	: 6
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp ibrs_enhanced fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat avx512_vnni md_clear arch_capabilities
bugs		: spectre_v1 spectre_v2 spec_store_bypass swapgs ta

In [5]:
!cat /proc/meminfo

MemTotal:       55533064 kB
MemFree:        46681580 kB
MemAvailable:   53578060 kB
Buffers:          479028 kB
Cached:          6840628 kB
SwapCached:            0 kB
Active:          1133436 kB
Inactive:        7119060 kB
Active(anon):       6660 kB
Inactive(anon):   933716 kB
Active(file):    1126776 kB
Inactive(file):  6185344 kB
Unevictable:          20 kB
Mlocked:              20 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               460 kB
Writeback:           120 kB
AnonPages:        932936 kB
Mapped:           387008 kB
Shmem:              7516 kB
KReclaimable:     212560 kB
Slab:             288008 kB
SReclaimable:     212560 kB
SUnreclaim:        75448 kB
KernelStack:        9952 kB
PageTables:        18860 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    27766532 kB
Committed_AS:    4512004 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       91416 kB
VmallocChunk:    

In [12]:
!nvidia-smi -L

GPU 0: NVIDIA L4 (UUID: GPU-6d15bab3-9be7-40ff-6f28-86e662b837b3)


In [11]:
!nvidia-smi

Sun May 11 04:48:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   34C    P8             11W /   72W |       3MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Preparation

In [1]:
import pandas as pd
import sys
import cornac
import tensorflow as tf
import recommenders

from recommenders.utils.timer import Timer
from recommenders.evaluation.python_evaluation import (
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
)

from ranx import Qrels, Run, evaluate

print(f"System version: {sys.version}")
print(f"Tensorflow version: {tf.version}")
print(f"Cornac version: {cornac.__version__}")
print(f"Recommenders version: {recommenders.__version__}")

System version: 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]
Tensorflow version: <module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.11/dist-packages/tensorflow/_api/v2/version/__init__.py'>
Cornac version: 2.3.0
Recommenders version: 1.2.1


In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import torch

print(f"{torch.__name__}: {torch.__version__}")

torch: 2.6.0+cu124


## GRU

In [4]:
DATA_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/ml-1m/data_split/'

In [5]:
train_data = pd.read_csv(f'{DATA_PATH}/data_train_full.tsv', sep='\t')
test_data = pd.read_csv(f'{DATA_PATH}/data_test.tsv', sep='\t')

In [6]:
len(train_data), len(test_data)

(665395, 166351)

In [7]:
# top k items to recommend
TOP_K = 20
NUM_EPOCHS = 20
SEED=100

In [8]:
from cornac.eval_methods import NextItemEvaluation

next_item_eval = NextItemEvaluation.from_splits(
    train_data=list(train_data[['user_id', 'movie_id', 'timestamp']].itertuples(index=False)),
    test_data=list(test_data[['user_id', 'movie_id', 'timestamp']].itertuples(index=False)),
    exclude_unknowns=True,
    verbose=True,
    fmt="SIT",
)

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 1
Number of items = 3571
Number of sessions = 5755
---
Test data:
Number of users = 1
Number of items = 3571
Number of sessions = 5755
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 1
Total items = 3571
Total sessions = 11510


In [14]:
from collections import OrderedDict, defaultdict
import numpy as np
from tqdm.notebook import tqdm
import itertools

def ranking_eval(
    model,
    train_set,
    test_set,
    exclude_unknowns=True,
    mode="last",
    verbose=False,
):

    rankings = []
    scores = []
    user_sessions = defaultdict(list)
    session_ids = []
    for [sid], [mapped_ids], [session_items] in tqdm(
        test_set.si_iter(batch_size=1, shuffle=False),
        total=len(test_set.sessions)):

        if len(session_items) < 2:  # exclude all session with size smaller than 2
            continue
        user_idx = test_set.uir_tuple[0][mapped_ids[0]]
        session_ids.append(sid)

        start_pos = 1 if mode == "next" else len(session_items) - 1
        for test_pos in range(start_pos, len(session_items), 1):
            test_pos_items = session_items[test_pos]

            # binary mask for ground-truth positive items
            u_gt_pos_mask = np.zeros(test_set.num_items, dtype="int")
            u_gt_pos_mask[test_pos_items] = 1

            # binary mask for ground-truth negative items, removing all positive items
            u_gt_neg_mask = np.ones(test_set.num_items, dtype="int")
            u_gt_neg_mask[test_pos_items] = 0

            # filter items being considered for evaluation
            if exclude_unknowns:
                u_gt_pos_mask = u_gt_pos_mask[: train_set.num_items]
                u_gt_neg_mask = u_gt_neg_mask[: train_set.num_items]

            u_gt_pos_items = np.nonzero(u_gt_pos_mask)[0]
            u_gt_neg_items = np.nonzero(u_gt_neg_mask)[0]
            item_indices = np.nonzero(u_gt_pos_mask + u_gt_neg_mask)[0]


            item_rank, item_scores = model.rank(
                user_idx,
                item_indices,
                history_items=session_items[:test_pos],
                history_mapped_ids=mapped_ids[:test_pos],
                sessions=test_set.sessions,
                session_indices=test_set.session_indices,
                extra_data=test_set.extra_data,
            )
            item_scores = item_scores[item_rank]
            item_rank = [key for value in item_rank for key, val in train_set.iid_map.items() if val == value]

            rankings.append(item_rank)
            scores.append(item_scores)

    return rankings, scores

## Default Model

In [ ]:
 gru = cornac.models.GRU4Rec(
        n_epochs=NUM_EPOCHS,
        device="cuda",
        verbose=True,
        batch_size=512,
        seed=SEED,
    )

In [ ]:
with Timer() as t:
    gru.fit(next_item_eval.train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/20 [00:00<?, ?it/s]

Took 182.3441 seconds for training.


In [ ]:
gru.transform(next_item_eval.test_set)

In [ ]:
gru_ranking, gru_scores = ranking_eval(
    gru,
    next_item_eval.train_set,
    next_item_eval.test_set,
    exclude_unknowns=True,
    mode="last",
    verbose=False,
)

  0%|          | 0/5755 [00:00<?, ?it/s]

In [ ]:
users = []
item = list(gru.train_set.iid_map.keys())
for uid, user_idx in tqdm(gru.train_set.sid_map.items()):
    user = [uid] * len(item)
    users.extend(user)

In [ ]:
flattened_ranking = list(itertools.chain(*gru_ranking))
flattened_score= list(itertools.chain(*gru_scores))

In [ ]:
len(users), len(flattened_ranking), len(flattened_score)

(20551105, 20551105, 20551105)

In [ ]:
print('a')

In [ ]:
df_gru_predictions = pd.DataFrame({'user_id':users, 'movie_id':flattened_ranking, 'prediction': flattened_score})

In [ ]:
ranking_metrics_gru = {
    'Precision@1' : precision_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="movie_id", col_rating='rating', k=1),
    'Precision@10' : precision_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="movie_id", col_rating='rating', k=10),
    'Precision@20' : precision_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="movie_id", col_rating='rating', k=20),
    'Recall@1' : recall_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="movie_id", col_rating='rating', k=1),
    'Recall@10' : recall_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="movie_id", col_rating='rating', k=10),
    'Recall@20' : recall_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="movie_id", col_rating='rating', k=20),
    'NDCG@1' : ndcg_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="movie_id", col_rating='rating', k=1),
    'NDCG@10' : ndcg_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="movie_id", col_rating='rating', k=10),
    'NDCG@20' : ndcg_at_k(test_data, df_gru_predictions, col_user="user_id", col_item="movie_id", col_rating='rating', k=20)
}

In [ ]:
pd.DataFrame(ranking_metrics_gru, index=['metrics']).T

,metrics
Precision@1,0.151868
Precision@10,0.121529
Precision@20,0.103041
Recall@1,0.011570
Recall@10,0.088847
Recall@20,0.146866
NDCG@1,0.151868
NDCG@10,0.139839
NDCG@20,0.151310


In [ ]:
gru.save('/content/drive/MyDrive/Final Project/Codes/animelist-goodreads-recommendation/ml-1m/model')

GRU4Rec model is saved to /content/drive/MyDrive/Final Project/Codes/animelist-goodreads-recommendation/ml-1m/model/GRU4Rec/2025-04-19_12-17-05-767934.pkl


'/content/drive/MyDrive/Final Project/Codes/animelist-goodreads-recommendation/ml-1m/model/GRU4Rec/2025-04-19_12-17-05-767934.pkl'

In [ ]:
test_data['user_id'] = test_data['user_id'].astype(str)
test_data['movie_id'] = test_data['movie_id'].astype(str)

df_gru_predictions['user_id'] = df_gru_predictions['user_id'].astype(str)
df_gru_predictions['movie_id'] = df_gru_predictions['movie_id'].astype(str)
df_gru_predictions['prediction'] = df_gru_predictions['prediction'].astype(float)

In [ ]:
qrels = Qrels.from_df(
    df=test_data,
    q_id_col="user_id",
    doc_id_col="movie_id",
    score_col="rating",
)

run = Run.from_df(
    df=df_gru_predictions,
    q_id_col="user_id",
    doc_id_col="movie_id",
    score_col="prediction",
)

In [ ]:
ranking_metrics_gru = pd.DataFrame({'GRU': evaluate(qrels, run, ["mrr", "ndcg@10", "recall@10", "precision@10", "hit_rate@10"])})

/usr/local/lib/python3.11/dist-packages/ranx/metrics/reciprocal_rank.py:29: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _reciprocal_rank(qrels[i], run[i], k, rel_lvl)


In [ ]:
ranking_metrics_gru

,GRU
mrr,0.315608
ndcg@10,0.122832
recall@10,0.088847
precision@10,0.121529
hit_rate@10,0.702172


## Hyperparameter Model

In [9]:
import pickle

In [10]:
gru = cornac.models.GRU4Rec(
            layers=[450],
            loss='bpr-max',
            learning_rate=0.0074,
            dropout_p_embed=0.1891,
            dropout_p_hidden=0.4621,
            batch_size=128,
            n_epochs=20,
            device="cuda",
            verbose=True,
            seed=SEED,
        )

In [11]:
with Timer() as t:
    gru.fit(next_item_eval.train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/20 [00:00<?, ?it/s]

Took 517.4843 seconds for training.


In [12]:
gru.transform(next_item_eval.test_set)

In [15]:
gru_ranking, gru_scores = ranking_eval(
    gru,
    next_item_eval.train_set,
    next_item_eval.test_set,
    exclude_unknowns=True,
    mode="last",
    verbose=False,
)

  0%|          | 0/5755 [00:00<?, ?it/s]

In [16]:
users = []
item = list(gru.train_set.iid_map.keys())
for uid, user_idx in gru.train_set.sid_map.items():
    user = [uid] * len(item)
    users.extend(user)

In [17]:
import itertools

flattened_ranking = list(itertools.chain(*gru_ranking))
flattened_score= list(itertools.chain(*gru_scores))

In [18]:
len(users), len(flattened_ranking), len(flattened_score)

(20551105, 20551105, 20551105)

In [19]:
df_gru_predictions = pd.DataFrame({'user_id':users, 'movie_id':flattened_ranking, 'prediction': flattened_score})

In [20]:
PREDICTION_PATH = '/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/ml-1m/predictions'
pickle.dump(df_gru_predictions, open(f'{PREDICTION_PATH}/predictions_gru_opt.pkl', 'wb'))

In [21]:
filtered_df_gru = df_gru_predictions.merge(train_data, on=['user_id', 'movie_id'], how='left', indicator=True)
filtered_df_gru = filtered_df_gru[filtered_df_gru['_merge'] == 'left_only']
filtered_df_gru = filtered_df_gru.drop(columns=['_merge'])

In [22]:
test_data['user_id'] = test_data['user_id'].astype(str)
test_data['movie_id'] = test_data['movie_id'].astype(str)

filtered_df_gru['user_id'] = filtered_df_gru['user_id'].astype(str)
filtered_df_gru['movie_id'] = filtered_df_gru['movie_id'].astype(str)
filtered_df_gru['prediction'] = filtered_df_gru['prediction'].astype(float)

In [24]:
qrels = Qrels.from_df(
    df=test_data,
    q_id_col="user_id",
    doc_id_col="movie_id",
    score_col="rating",
)

run = Run.from_df(
    df=filtered_df_gru,
    q_id_col="user_id",
    doc_id_col="movie_id",
    score_col="prediction",
)

In [25]:
ranking_metrics_gru = pd.DataFrame({'GRU': evaluate(qrels, run, ["mrr", "ndcg@10", "recall@10", "precision@10", "hit_rate@10"])})

/usr/local/lib/python3.11/dist-packages/ranx/metrics/reciprocal_rank.py:29: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _reciprocal_rank(qrels[i], run[i], k, rel_lvl)


In [26]:
ranking_metrics_gru

,GRU
mrr,0.555957
ndcg@10,0.246530
recall@10,0.165938
precision@10,0.225856
hit_rate@10,0.926325


In [27]:
gru.save("/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/ml-1m/model")

GRU4Rec model is saved to /content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/ml-1m/model/GRU4Rec/2025-05-11_09-50-21-126377.pkl


'/content/drive/MyDrive/Final Project/Codes/animelist-goodbooks-recommendation/ml-1m/model/GRU4Rec/2025-05-11_09-50-21-126377.pkl'